In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import requests
import bs4
from bs4 import BeautifulSoup
import time

In [6]:
qualifying_results = pd.DataFrame()
for year in list(range(1983,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)

    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        try:
            this_link = new_url.format(link)
            df = pd.read_html(new_url.format(link))
            df = df[0]
            df['season'] = year
            df['round'] = n+1
            for col in df:
                if 'Unnamed' in col:
                    df.drop(col, axis = 1, inplace = True)

            year_df = pd.concat([year_df, df])
        except:
            print(link)

    qualifying_results = pd.concat([qualifying_results, year_df])
    
print(qualifying_results.shape)

/en/results.html/2021/races/1105/qatar/starting-grid.html
(15313, 7)


In [7]:
qualifying_results.head()

,Pos,No,Driver,Car,Time,season,round
0,1,1,Keke Rosberg ROS,Williams Honda,1:34.526,1983,1
1,2,15,Alain Prost PRO,Renault,1:34.672,1983,1
2,3,27,Patrick Tambay TAM,Ferrari,1:34.758,1983,1
3,4,5,Nelson Piquet PIQ,Brabham BMW,1:35.114,1983,1
4,5,35,Derek Warwick WAR,Toleman Hart,1:35.206,1983,1


In [8]:
qualifying_results.tail()

,Pos,No,Driver,Car,Time,season,round
14,15,5,Sebastian Vettel VET,Aston Martin Mercedes,1:24.305,2021,22
15,16,6,Nicholas Latifi LAT,Williams Mercedes,1:24.338,2021,22
16,17,63,George Russell RUS,Williams Mercedes,1:24.423,2021,22
17,18,7,Kimi Räikkönen RAI,Alfa Romeo Racing Ferrari,1:24.779,2021,22
18,19,47,Mick Schumacher MSC,Haas Ferrari,1:24.906,2021,22


In [9]:
qualifying_results.rename(columns = {'Pos': 'grid_position', 'Driver': 'driver_name', 'Car': 'car',
                                     'Time': 'qualifying_time'}, inplace = True)

In [10]:
qualifying_results.drop('No', axis = 1, inplace = True)

In [11]:
qualifying_results.to_csv('qualifying.csv', index = False)